In [2]:
import os
from pathlib import Path

import ee
import geemap
import numpy as np
import shapely

from nad.load import load_equip_df

In [3]:
ee.Initialize(project="ee-ursa-test")

In [4]:
data_path = Path(os.environ["DATA_PATH"])

In [5]:
df = (
    load_equip_df(
        data_path / "datos" / "pparquesWgs84",
        extra_cols=["CLASIFICAC"],
    )
    .assign(geometry=lambda df: df["geometry"].make_valid())
    .to_crs("EPSG:4326")
)

df = df[df["geometry"].geom_type == "Polygon"].reset_index(names="polygon_id")

bounds = ee.geometry.Geometry.Polygon(
    list(shapely.box(*df.total_bounds).exterior.coords),
)

In [6]:
chunk_size = 4000
df_chunks = np.array_split(df, 2)

feature_chunks = [geemap.geopandas_to_ee(chunk) for chunk in df_chunks]

/Users/rodolfofigueroa/Documents/NAD/.venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoDataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/rodolfofigueroa/Documents/NAD/.venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoDataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/rodolfofigueroa/Documents/NAD/.venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoDataFrame.transpose' instead.
  return bound(*args, **kwds)


In [7]:
img_canopy = (
    ee.ImageCollection("projects/sat-io/open-datasets/facebook/meta-canopy-height")
    .filterBounds(bounds)
    .first()
)

mask = img_canopy.gte(3).unmask(0)

In [13]:
out = ee.data.computeFeatures(
    {
        "expression": ee.image.Image.pixelArea()
        .multiply(mask)
        .reduceRegions(
            feature_chunks[0],
            reducer=ee.reducer.Reducer.sum(),
            scale=1,
            crs="EPSG:4326",
        ),
        "fileFormat": "GEOPANDAS_GEODATAFRAME",
    },
)

out = (
    out.set_crs("EPSG:4326")
    .to_crs("EPSG:6372")
    .assign(
        area_frac=lambda df: df["sum"] / df["geometry"].area.sum(),
    )
    .rename(columns={"sum": "area"})
)

In [31]:
out.query("area == 0")["CLASIFICAC"].value_counts()

CLASIFICAC
CAMELLON      492
PARQUE        210
AREA VERDE    123
GLORIETA       80
Name: count, dtype: int64